# Simple Self Contained Model of the Funcspec Project

## Imports

In [ ]:
import torch.nn as nn
import torch
import numpy as np
from tqdm.notebook import tqdm as tqdm_n
import matplotlib.pyplot as plt
import seaborn as sns


## Data

### Datasets

In [ ]:


letter_datasets = 

### Tasks